# Practical example. Audiobooks

## Problem

You are given data from an Audiobook app. Logically, it relates only to the audio versions of books. Each customer in the database has made a purchase at least once, that's why he/she is in the database. We want to create a machine learning algorithm based on our available data that can predict if a customer will buy again from the Audiobook company.

The main idea is that if a customer has a low probability of coming back, there is no reason to spend any money on advertizing to him/her. If we can focus our efforts ONLY on customers that are likely to convert again, we can make great savings. Moreover, this model can identify the most important metrics for a customer to come back again. Identifying new customers creates value and growth opportunities.

You have a .csv summarizing the data. There are several variables: Customer ID, Book length in mins_avg (average of all purchases), Book length in minutes_sum (sum of all purchases), Price Paid_avg (average of all purchases), Price paid_sum (sum of all purchases), Review (a Boolean variable), Review (out of 10), Total minutes listened, Completion (from 0 to 1), Support requests (number), and Last visited minus purchase date (in days).

So these are the inputs (excluding customer ID, as it is completely arbitrary. It's more like a name, than a number).

The targets are a Boolean variable (so 0, or 1). We are taking a period of 2 years in our inputs, and the next 6 months as targets. So, in fact, we are predicting if: based on the last 2 years of activity and engagement, a customer will convert in the next 6 months. 6 months sounds like a reasonable time. If they don't convert after 6 months, chances are they've gone to a competitor or didn't like the Audiobook way of digesting information. 

The task is simple: create a machine learning algorithm, which is able to predict if a customer will buy again. 

This is a classification problem with two classes: won't buy and will buy, represented by 0s and 1s. 

Good luck!

## Create the machine learning algorithm



### Import the relevant libraries

In [1]:
import numpy as np
import tensorflow as tf

### Data

In [2]:
# dataset is stored under 2-tuple 
npz = np.load("Audiobooks_data_train.npz")

# typepo of the key tragets!!!
train_inputs = npz['inputs'].astype(np.float64)
train_targets = npz['targets'].astype(np.int64)

In [3]:
npz = np.load("Audiobooks_data_validation.npz")

validation_inputs, validation_targets = npz['inputs'].astype(np.float64), npz['targets'].astype(np.int64)

In [4]:
npz = np.load("Audiobooks_data_test.npz")

test_inputs, test_targets = npz['inputs'].astype(np.float64), npz['targets'].astype(np.int64)

In [5]:
# let's create a temporary variable npz, where we will store each of the three Audiobooks datasets
npz = np.load('Audiobooks_data_train.npz')

# we extract the inputs using the keyword under which we saved them
# to ensure that they are all floats, let's also take care of that
train_inputs = npz['inputs'].astype(float)
# targets must be int because of sparse_categorical_crossentropy (we want to be able to smoothly one-hot encode them)
train_targets = npz['targets'].astype(int)

# we load the validation data in the temporary variable
npz = np.load('Audiobooks_data_validation.npz')
# we can load the inputs and the targets in the same line
validation_inputs, validation_targets = npz['inputs'].astype(float), npz['targets'].astype(int)

# we load the test data in the temporary variable
npz = np.load('Audiobooks_data_test.npz')
# we create 2 variables that will contain the test inputs and the test targets
test_inputs, test_targets = npz['inputs'].astype(float), npz['targets'].astype(int)

In [6]:
print(train_inputs.shape, train_targets.shape)
print(validation_inputs.shape, validation_targets.shape)
print(test_inputs.shape, test_targets.shape)

(3579, 10) (3579,)
(447, 10) (447,)
(448, 10) (448,)


### Model 
    outline, optimize, layers, loss, early stop and training

In [16]:
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                            ])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 100

max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs, 
          train_targets, 
          batch_size=batch_size, 
          epochs=max_epochs,
          validation_data=(validation_inputs, validation_targets),
          callbacks= [early_stopping],
          verbose=2
         )

Epoch 1/100
36/36 - 2s - loss: 0.5907 - accuracy: 0.6739 - val_loss: 0.5009 - val_accuracy: 0.7651 - 2s/epoch - 50ms/step
Epoch 2/100
36/36 - 0s - loss: 0.4574 - accuracy: 0.7628 - val_loss: 0.4610 - val_accuracy: 0.7740 - 186ms/epoch - 5ms/step
Epoch 3/100
36/36 - 0s - loss: 0.4090 - accuracy: 0.7807 - val_loss: 0.4293 - val_accuracy: 0.7785 - 181ms/epoch - 5ms/step
Epoch 4/100
36/36 - 0s - loss: 0.3880 - accuracy: 0.7946 - val_loss: 0.4377 - val_accuracy: 0.7830 - 179ms/epoch - 5ms/step
Epoch 5/100
36/36 - 0s - loss: 0.3800 - accuracy: 0.8027 - val_loss: 0.4160 - val_accuracy: 0.7919 - 189ms/epoch - 5ms/step
Epoch 6/100
36/36 - 0s - loss: 0.3714 - accuracy: 0.8072 - val_loss: 0.4328 - val_accuracy: 0.8076 - 186ms/epoch - 5ms/step
Epoch 7/100
36/36 - 0s - loss: 0.3688 - accuracy: 0.8019 - val_loss: 0.4182 - val_accuracy: 0.7852 - 187ms/epoch - 5ms/step


### Test Model

In [17]:
test_loss, test_accuracy = model.evaluate(x=test_inputs, y=test_targets)

14/14 [==============================] - 0s 3ms/step - loss: 0.3453 - accuracy: 0.8147


In [22]:
print('Test Loss : {0:.2f} Test accuracy : {1:0.2f}%'.format(test_loss, test_accuracy*100))

Test Loss : 0.35 Test accuracy : 81.47%
